In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

In [3]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
test_data['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [6]:
features = ['Pclass','Sex','Age','SibSp','Parch','Embarked']
# creating bool series True for NaN values 
bool_seriesTr = pd.isnull(train_data['Age']).sum()
avgAgeTr=train_data['Age'].sum()/(train_data['Age'].count()-bool_seriesTr)
bool_seriesTe = pd.isnull(test_data['Age']).sum()
avgAgeTe=test_data['Age'].sum()/(test_data['Age'].count()-bool_seriesTe)
#61,829
# filtering data 
# displaying data 

train_data["Age"].fillna(avgAgeTr, inplace = True)
train_data["Embarked"].fillna('S',inplace = True)
test_data["Age"].fillna(avgAgeTe, inplace = True)
test_data["Embarked"].fillna('S',inplace = True)

In [7]:
y_train = train_data["Survived"]
features = ['Pclass','Sex','Age','SibSp','Parch','Embarked']
X_train = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

In [8]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
parameters = {
    "n_estimators":[5,10,50,100,250,500],
    "max_depth":[2,3,4,5,8,10,16,32,None]
    
}

In [9]:
from sklearn.model_selection import GridSearchCV
cv = GridSearchCV(rfc,parameters,cv=5)
cv.fit(X_train,y_train.values.ravel())

def display(results):
    print(f'Best parameters are: {results.best_params_}')
    print("\n")
    mean_score = results.cv_results_['mean_test_score']
    std_score = results.cv_results_['std_test_score']
    params = results.cv_results_['params']
    for mean,std,params in zip(mean_score,std_score,params):
        print(f'{round(mean,3)} + or -{round(std,3)} for the {params}')

In [10]:
display(cv)

Best parameters are: {'max_depth': 4, 'n_estimators': 500}


0.797 + or -0.022 for the {'max_depth': 2, 'n_estimators': 5}
0.79 + or -0.023 for the {'max_depth': 2, 'n_estimators': 10}
0.789 + or -0.021 for the {'max_depth': 2, 'n_estimators': 50}
0.788 + or -0.02 for the {'max_depth': 2, 'n_estimators': 100}
0.789 + or -0.021 for the {'max_depth': 2, 'n_estimators': 250}
0.787 + or -0.019 for the {'max_depth': 2, 'n_estimators': 500}
0.806 + or -0.01 for the {'max_depth': 3, 'n_estimators': 5}
0.81 + or -0.019 for the {'max_depth': 3, 'n_estimators': 10}
0.809 + or -0.014 for the {'max_depth': 3, 'n_estimators': 50}
0.809 + or -0.017 for the {'max_depth': 3, 'n_estimators': 100}
0.805 + or -0.019 for the {'max_depth': 3, 'n_estimators': 250}
0.81 + or -0.015 for the {'max_depth': 3, 'n_estimators': 500}
0.816 + or -0.021 for the {'max_depth': 4, 'n_estimators': 5}
0.816 + or -0.019 for the {'max_depth': 4, 'n_estimators': 10}
0.818 + or -0.018 for the {'max_depth': 4, 'n_estimators': 

In [11]:
# Best parameters are: {'max_depth': 5, 'n_estimators': 500}
model = RandomForestClassifier(n_estimators=500, max_depth=5, random_state=1)

model.fit(X_train,y_train)

predictions = model.predict(X_test)

output = pd.DataFrame({"PassengerId":test_data["PassengerId"],"Survived":predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")


Your submission was successfully saved!
